In [4]:
!apt-get update
!apt-get install -y mpich
!pip install mpi4py

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mpich is already the newest version (4.0-3).
0 upgraded, 0 newly installed, 0 to remove and 63

In [5]:
%%writefile mpi_master_worker.py
from mpi4py import MPI

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    # Master process
    print(f"Master process started. Expecting {size-1} worker messages...\n")

    for i in range(1, size):
        message = comm.recv(source=i)
        print(
            f"Received from process {i} | "
            f"Task: {message['task']} | "
            f"Result: {message['result']}"
        )

    print("\nAll worker results received.")

else:
    # Worker processes
    data_chunk = list(range(rank * 10, rank * 10 + 10))
    computed_sum = sum(data_chunk)

    message = {
        "rank": rank,
        "task": f"Data chunk {rank}",
        "result": computed_sum
    }

    comm.send(message, dest=0)


Overwriting mpi_master_worker.py


In [9]:
!mpiexec --allow-run-as-root --oversubscribe -n 4 python mpi_master_worker.py

Master process started. Expecting 3 worker messages...

Received from process 1 | Task: Data chunk 1 | Result: 145
Received from process 2 | Task: Data chunk 2 | Result: 245
Received from process 3 | Task: Data chunk 3 | Result: 345

All worker results received.
